## Quality assurance when you have fully labelled data

In this example, our data contains a fully-populated ground-truth column called `cluster` that enables us to perform accuracy analysis of the final model

In [1]:
from splink.datasets import splink_datasets
import altair as alt
alt.renderers.enable("html")

df = splink_datasets.fake_1000

df.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net,0
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net,0


In [2]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

In [3]:
linker = DuckDBLinker(df, settings, set_up_basic_logging=False)
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


In [4]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=5)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(block_on("email"))

In [6]:
linker.truth_space_table_from_labels_column(
    "cluster", match_weight_round_to_nearest=0.1
).as_pandas_dataframe(limit=5)

,truth_threshold,match_probability,row_count,p,n,tp,tn,fp,fn,P_rate,N_rate,tp_rate,tn_rate,fp_rate,fn_rate,precision,recall,f1
0,-24.3,4.841400e-08,4353.0,2031.0,2322.0,2031.0,0.0,2322.0,0.0,0.466575,0.533425,1.000000,0.000000,1.000000,0.000000,0.466575,1.000000,0.636278
1,-23.8,6.846774e-08,4353.0,2031.0,2322.0,2030.0,0.0,2322.0,1.0,0.466575,0.533425,0.999508,0.000000,1.000000,0.000492,0.466452,0.999508,0.636065
2,-23.7,7.338190e-08,4353.0,2031.0,2322.0,2030.0,234.0,2088.0,1.0,0.466575,0.533425,0.999508,0.100775,0.899225,0.000492,0.492958,0.999508,0.660270
3,-22.6,1.572975e-07,4353.0,2031.0,2322.0,2030.0,428.0,1894.0,1.0,0.466575,0.533425,0.999508,0.184324,0.815676,0.000492,0.517329,0.999508,0.681780
4,-22.5,1.685873e-07,4353.0,2031.0,2322.0,2030.0,583.0,1739.0,1.0,0.466575,0.533425,0.999508,0.251077,0.748923,0.000492,0.538604,0.999508,0.700000


In [7]:
linker.roc_chart_from_labels_column("cluster")

alt.Chart(...)

In [8]:
linker.precision_recall_chart_from_labels_column("cluster")

alt.Chart(...)

In [9]:
# Plot some false positives
linker.prediction_errors_from_labels_column(
    "cluster", include_false_negatives=True, include_false_positives=True
).as_pandas_dataframe(limit=5)

,clerical_match_score,found_by_blocking_rules,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,...,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,cluster_l,cluster_r,match_key
0,1.0,True,-2.430705,0.156456,248,249,Joshua,Joshua,4,85.509553,...,NaN,1.000000,1.000000,NaN,j.williams@levine-johnson.com,-1,1.0,64,64,0
1,1.0,True,-0.104231,0.481946,361,362,Mohammed,Mohammed,4,85.509553,...,NaN,1.000000,1.000000,NaN,mohammedfox24@wilson.com,-1,1.0,95,95,0
2,1.0,True,-2.430705,0.156456,376,380,Eliza,Eliza,4,85.509553,...,NaN,1.000000,1.000000,NaN,elizataylor@marshall.com,-1,1.0,98,98,0
3,1.0,True,-0.948461,0.341319,599,602,Toby,Toby,4,85.509553,...,0.212792,10.780255,0.259162,NaN,NaN,-1,1.0,148,148,0
4,1.0,True,-0.948461,0.341319,600,602,Toby,Toby,4,85.509553,...,0.212792,10.780255,0.259162,toby.d@menhez.com,NaN,-1,1.0,148,148,0


In [10]:
records = linker.prediction_errors_from_labels_column(
    "cluster", include_false_negatives=True, include_false_positives=True
).as_record_dict(limit=5)

linker.waterfall_chart(records)

alt.LayerChart(...)